In [75]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
import yagmail

nav = webdriver.Chrome()

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

#Usuário e Senha para enviar o e-mail
usuario = yagmail.SMTP(user='xxx@xxx.com', password='xxxxxxxxx')

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


## Definir funcões ##

In [76]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # Tratar os valores  ##
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)

    # Navegar até o Google Shopping e realizar a busca  ##
    nav.get("https://shopping.google.com.br/")
    nav.find_element(By.XPATH, '/html/body/c-wiz[1]/div/div/c-wiz/form/div[2]/div[1]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/c-wiz[1]/div/div/c-wiz/form/div[2]/div[1]/input').send_keys(Keys.ENTER)

    # Pegar a lista de resultados  ##
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    
    # Verificar se todas as informacões da busca no Google Shopping estão batendo com a base de dados ##
    lista_ofertas = []
    
    for resultado in lista_resultados:
        try:
            nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
            nome = nome.lower()

            ## Verificacao do nome ##
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True

            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False

            ## Verificacao do preco ##
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)

                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
                
    return lista_ofertas


def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # Tratar os valores  ##
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)

    # Navegar até o Buscapé e realizar a busca  ##

    nav.get("https://www.buscape.com.br/")
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[1]/input').send_keys(produto, Keys.ENTER)

    # Pegar a lista de resultados  ##
    time.sleep(10)
    lista_resultados_buscape = nav.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')

    # Verificar se todas as informacões da busca no Buscapé estão batendo com a base de dados ##
    lista_ofertas = []

    for resultado in lista_resultados_buscape:
        try:
            nome = resultado.find_element(By.CLASS_NAME, 'Cell_Name__pxLaW').text
            nome = nome.lower()

            ## Verificacao do nome ##
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True

            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False

            ## Verificacao do preco ##
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__JXsj_').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    link = resultado.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
                
    return lista_ofertas

## Contrucão da lista de ofertas

In [77]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preco', 'Link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
    
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preco', 'Link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None

## Exportar a base de ofertas para Excel

In [78]:
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel("Ofertas.xlsx", index=False)

## Enviar por e-mail

In [83]:
corpo_email = '{}'.format(tabela_ofertas.to_html(index=False))

## Devido bug no yagmail é necessário trocar toda quebra de linha "\n" por "" para não termos  ##
##  linhas em branco no corpo do email.                                                        ##
corpo_email = corpo_email.replace("\n", "")

if len(tabela_ofertas.index) > 0:
    usuario.send(to='raps_rnb@hotmail.com',
                 subject='Tabela de ofertas',
                 contents= corpo_email, 
                 attachments=['Ofertas.xlsx']
                )
nav.quit()